In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from collections import defaultdict 
from functools import cmp_to_key
from scipy.optimize import nnls

import glob
import torch
import numpy as np
from skimage import io, transform

import torchvision.transforms.functional as F 
from torch.utils.data import Dataset
from PIL import Image
import random

import itertools 
import torch.nn as nn
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import pandas as pd
from torch.utils.data import DataLoader
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
class Graph: 
    def __init__(self,vertices): 
        self.V= vertices            
        self.graph = defaultdict(list)  
 
    def addEdge(self,u,v): 
        self.graph[u].append(v) 
   
    def printPathsFunc(self, u, d, visited, path, current_path_list): 
        visited[u]= True
        path.append(u) 

        if u == d: 
            path_copy = path[:]
            current_path_list.append(path_copy)
        else: 
            for i in self.graph[u]: 
                if visited[i]==False: 
                    self.printPathsFunc(i, d, visited, path, current_path_list) 
                      
        path.pop() 
        visited[u]= False
        return current_path_list

        
    def printPaths(self, s, d): 
        total_results = []
        for start in s:
            for dest in d:
                path = []
                visited =[False]*(self.V) 
                current_path_list = []
                current_path_results = self.printPathsFunc(start, dest, visited, path, current_path_list) 
                if len(current_path_results) != 0:
                    total_results.extend(current_path_results)
        return total_results
    
    
def graph_process(config_path):
    cluster_unify = []
    partial_order = []
    start_candidates = []
    end_candidates = []
    
    f = open(config_path, 'r')
    while True:
        line = f.readline()
        if '=' in line:
            unify = list(map(int, line.split('=')))
            cluster_unify.append(unify)
        elif '<' in line:
            order = list(map(int, line.split('<')))
            partial_order.append(order)
            start_candidates.append(order[0])
            end_candidates.append(order[1])
            
        if not line: break
    f.close()
        
    start = []
    end = []
    for element in start_candidates:
        if element in end_candidates:
            continue
        start.append(element)
    
    for element in end_candidates:
        if element in start_candidates:
            continue
        end.append(element)
    
    start = list(set(start))
    end = list(set(end))
    return start, end, partial_order, cluster_unify



def generate_graph(partial_order_list, vertex_num):
    cluster_graph = Graph(vertex_num)
    for pair in partial_order_list:
        cluster_graph.addEdge(pair[0], pair[1])
    return cluster_graph 


def save_graph_config(ordered_list, name):
    f = open("/home/haoying/res_zl12_effnet_v4/{}".format(name), 'w')
        
    for i in range(len(ordered_list) - 1):
        f.write('{}<{}\n'.format(ordered_list[i+1][0], ordered_list[i][0]))
    
    for orders in ordered_list:
        if len(orders) >= 2:
            f.write(str(orders[0]))
            for element in orders[1:]:
                f.write('={}'.format(element))
            f.write('\n')        
    f.close()        
    
            
def graph_inference_nightlight(grid_df, nightlight_df, cluster_num, file_path):
    def numeric_compare(x, y):
        pop_list1 = df_merge_group.get_group(x)['nightlights'].tolist()
        pop_list2 = df_merge_group.get_group(y)['nightlights'].tolist()
        tTestResult = stats.ttest_ind(pop_list1, pop_list2)
        if (tTestResult.pvalue < 0.01) and (np.mean(pop_list1) < np.mean(pop_list2)):
            return 1
        elif (tTestResult.pvalue < 0.01) and (np.mean(pop_list1) >= np.mean(pop_list2)):
            return -1
        else:
            return 0
        
    df_merge = pd.merge(nightlight_df, grid_df, how='left', on='y_x')
    df_merge = df_merge.dropna()
    df_merge_group = df_merge.groupby('cluster_id')
    
    sorted_list = sorted(range(cluster_num - 1), key=cmp_to_key(numeric_compare))
    ordered_list = []
    ordered_list.append([sorted_list[0]])
    curr = 0
    for i in range(len(sorted_list) - 1):
        if numeric_compare(sorted_list[i], sorted_list[i+1]) == 0:
            ordered_list[curr].append(sorted_list[i+1])
        else:
            curr += 1
            ordered_list.append([sorted_list[i+1]])
            
    ordered_list.append([cluster_num - 1])        
    save_graph_config(ordered_list, file_path)
    return ordered_list

In [3]:
# L = []
# # cluster_num = 15
# file = list(pd.read_csv("/home/haoying/res_zl12_effnet/data/{}/cluster.csv".format(str(cluster_num)))['y_x'].values)
# L.extend(file)

In [3]:
class ClusterDataset(Dataset):
    def __init__(self, cluster_list, transform=None):
        self.file_list = []
        self.transform = transform      
        for cluster_num in cluster_list:
            file=list(pd.read_csv("/home/haoying/res_zl12_effnet_v4/data/{}/cluster.csv".format(str(cluster_num)))['y_x'].values)
            self.file_list.extend(file)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        image = Image.open("/home/haoying/data_zl12/{}".format(self.file_list[idx])).convert('RGB')
        if self.transform:
            image = self.transform(image).squeeze()
        return image

    
class RandomRotate(object):
    def __call__(self, images):
        rotated = np.stack([self.random_rotate(x) for x in images])
        return rotated
    
    def random_rotate(self, image):
        rand_num = np.random.randint(0, 4)
        if rand_num == 0:
            return np.rot90(image, k=1, axes=(0, 1))
        elif rand_num == 1:
            return np.rot90(image, k=2, axes=(0, 1))
        elif rand_num == 2:
            return np.rot90(image, k=3, axes=(0, 1))   
        else:
            return image
    
    
class Normalize(object):
    def __init__(self, mean, std, inplace=False):
        self.mean = mean
        self.std = std
        self.inplace = inplace

    def __call__(self, images):
        normalized = np.stack([F.normalize(x, self.mean, self.std, self.inplace) for x in images]) 
        return normalized
 

    
class Grayscale(object):
    def __init__(self, prob = 1):
        self.prob = prob

    def __call__(self, images):     
        random_num = np.random.randint(100, size=1)[0]
        if random_num <= self.prob * 100:
            gray_images = (images[:, 0, :, :] + images[:, 1, :, :] + images[:, 2, :, :]) / 3
            gray_scaled = gray_images.unsqueeze(1).repeat(1, 3, 1, 1)
            return gray_scaled
        else:
            return images

    

class ToTensor(object):
    def __call__(self, images):
        images = images.transpose((0, 3, 1, 2))
        return torch.from_numpy(images).float()

class AverageMeter(object):
    def __init__(self):
        self.reset()
                   
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0 

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
def make_data_loader(cluster_list, batch_sz):
    cluster_dataset = ClusterDataset(cluster_list, transform = transforms.Compose([                    
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                    ]))
    cluster_loader = torch.utils.data.DataLoader(cluster_dataset, batch_size=batch_sz, shuffle=True, num_workers=0, drop_last=True)
    return cluster_loader



def generate_loader_dict(total_list, batch_sz):
    loader_dict = {}
    for cluster_id in total_list:
        cluster_loader = make_data_loader([cluster_id], batch_sz)
        loader_dict[cluster_id] = cluster_loader        
    
#     for cluster_tuple in unified_cluster_list:
#         cluster_loader = make_data_loader(cluster_tuple, batch_sz)
#         for cluster_num in cluster_tuple:
#             loader_dict[cluster_num] = cluster_loader
    return loader_dict


def deactivate_batchnorm(model):
    for layer in [model.layer1,model.layer2,model.layer3,model.layer4]:
        for m in model.layer1:
            if isinstance(m, nn.BatchNorm2d):
                m.reset_parameters()
                m.eval()
                with torch.no_grad():
                    m.weight.fill_(1.0)
                    m.bias.zero_()
                
def train(epoch, model, optimizer, loader_list, cluster_path_list, device, batch_sz):
    model.train()
    # Deactivate the batch normalization before training
#     deactivate_batchnorm(model)
    
    train_loss = AverageMeter()
    reg_loss = AverageMeter()
    
    # For each cluster route
    path_idx = 0
    avg_loss = 0
    count = 0
    for cluster_path in cluster_path_list:
        path_idx += 1
        dataloaders = []
        for cluster_id in cluster_path:
            dataloaders.append(loader_list[cluster_id])
    
 
        for batch_idx, data in enumerate(zip(*dataloaders)):
            cluster_num = len(data)
            data_zip = torch.cat(data, 0).to(device)

            # Generating Score
            scores = model(data_zip).squeeze()
            score_list = torch.split(scores, batch_sz, dim = 0)
            
            # Standard deviation as a loss
            loss_var = torch.zeros(1).to(device)
            for score in score_list:
                loss_var += score.var()
            loss_var /= len(score_list)
            
            # Differentiable Ranking with sigmoid function
            rank_matrix = torch.zeros((batch_sz, cluster_num, cluster_num)).to(device)
            for itertuple in list(itertools.permutations(range(cluster_num), 2)):
                score1 = score_list[itertuple[0]]
                score2 = score_list[itertuple[1]]
                diff = 30 * (score2 - score1)
                results = torch.sigmoid(diff)
                rank_matrix[:, itertuple[0], itertuple[1]] = results
                rank_matrix[:, itertuple[1], itertuple[0]] = 1 - results

            rank_predicts = rank_matrix.sum(1)
            temp = torch.Tensor(range(cluster_num))
            target_rank = temp.unsqueeze(0).repeat(batch_sz, 1).to(device)

            # Equivalent to spearman rank correlation loss
            loss_train = ((rank_predicts - target_rank)**2).mean()
            loss = loss_train + loss_var * 4
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss.update(loss_train.item(), batch_sz)
            reg_loss.update(loss_var.item(), batch_sz)
            avg_loss += loss.item()
            count += 1

            # Print status
            if batch_idx % 10 == 0:
                print('Epoch: [{epoch}][{path_idx}][{elps_iters}] '
                      'Train loss: {train_loss.val:.4f} ({train_loss.avg:.4f}) '
                      'Reg loss: {reg_loss.val:.4f} ({reg_loss.avg:.4f})'.format(
                          epoch=epoch, path_idx=path_idx, elps_iters=batch_idx, train_loss=train_loss, reg_loss=reg_loss))
                
    return avg_loss / count

In [5]:
torch.manual_seed(15670107)
torch.cuda.manual_seed_all(15670107)
np.random.seed(15670107)
cluster_number = 19

grid_df = pd.read_csv('/home/haoying/res_zl12_effnet_v4/data/unified.csv')
nightlight_df = pd.read_csv('/home/haoying/res_zl12_effnet_v3/nightlights_labeled.csv')
graph_config = graph_inference_nightlight(grid_df, nightlight_df, cluster_number, 'CNconfig')   
graph_config

[[2],
 [0],
 [1],
 [3, 5, 8],
 [4, 10],
 [6, 7],
 [9],
 [12],
 [15],
 [14],
 [13],
 [16],
 [11, 17],
 [18]]

In [6]:
batch_size = 5

loader_list = generate_loader_dict(range(cluster_number), batch_size)
cluster_path_list=[]
for item in itertools.product(*graph_config):
    a=list(item)[::-1]
    cluster_path_list.append(a)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=torch.load('/home/haoying/res_zl12_effnet_v4/res_pretrained.pt')
# model._fc = nn.Sequential()
model._fc = nn.Sequential(nn.Linear(1280, 1))
model._swish = nn.Identity()
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [9]:
# model=torch.load('/home/haoying/res_zl12_resnet18/res_pretrained.pt')
# model

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
best_loss = float('inf')

In [ ]:
for epoch in range(50):          
    loss = train(epoch, model, optimizer, loader_list, cluster_path_list, device, batch_size)
    if epoch % 5 == 0 and epoch != 0:                
        if best_loss > loss:
            torch.save(model, '/home/haoying/res_zl12_effnet_v4/predict-unlimited.pt')
            best_loss = loss
            print("best loss: %.4f\n" % (best_loss))

Epoch: [0][1][0] Train loss: 46.5795 (46.5795) Reg loss: 0.5250 (0.5250)
Epoch: [0][1][10] Train loss: 6.1186 (17.9217) Reg loss: 1.3170 (0.9479)
Epoch: [0][1][20] Train loss: 9.8430 (13.1910) Reg loss: 1.6184 (1.2120)
Epoch: [0][1][30] Train loss: 9.8378 (11.5305) Reg loss: 1.0280 (1.2605)
Epoch: [0][1][40] Train loss: 8.1140 (10.6471) Reg loss: 0.8284 (1.1716)
Epoch: [0][1][50] Train loss: 9.0785 (10.2454) Reg loss: 0.9161 (1.1292)
Epoch: [0][1][60] Train loss: 6.7563 (9.9064) Reg loss: 0.9766 (1.1127)
Epoch: [0][1][70] Train loss: 6.3789 (9.4729) Reg loss: 0.9237 (1.1062)
Epoch: [0][1][80] Train loss: 6.7892 (9.1111) Reg loss: 0.7929 (1.0849)
Epoch: [0][1][90] Train loss: 6.1208 (8.9939) Reg loss: 0.5921 (1.0816)
Epoch: [0][1][100] Train loss: 5.8611 (8.7636) Reg loss: 0.8090 (1.0636)
Epoch: [0][1][110] Train loss: 7.2767 (8.6138) Reg loss: 0.9160 (1.0490)
Epoch: [0][1][120] Train loss: 5.9997 (8.4293) Reg loss: 0.9839 (1.0293)
Epoch: [0][1][130] Train loss: 5.8343 (8.3368) Reg loss

Epoch: [0][7][120] Train loss: 3.8472 (6.2825) Reg loss: 0.3746 (0.6800)
Epoch: [0][7][130] Train loss: 3.7665 (6.2610) Reg loss: 0.5117 (0.6774)
Epoch: [0][7][140] Train loss: 4.5592 (6.2406) Reg loss: 0.5746 (0.6751)
Epoch: [0][7][150] Train loss: 5.8524 (6.2260) Reg loss: 0.5044 (0.6730)
Epoch: [0][7][160] Train loss: 3.7603 (6.2075) Reg loss: 0.2931 (0.6702)
Epoch: [0][8][0] Train loss: 6.9700 (6.2071) Reg loss: 0.3837 (0.6697)
Epoch: [0][8][10] Train loss: 5.1273 (6.1989) Reg loss: 0.2837 (0.6670)
Epoch: [0][8][20] Train loss: 6.7339 (6.2047) Reg loss: 0.5116 (0.6651)
Epoch: [0][8][30] Train loss: 3.4656 (6.1917) Reg loss: 0.2991 (0.6626)
Epoch: [0][8][40] Train loss: 5.2063 (6.1857) Reg loss: 0.4128 (0.6602)
Epoch: [0][8][50] Train loss: 4.5179 (6.1780) Reg loss: 0.2817 (0.6575)
Epoch: [0][8][60] Train loss: 4.9870 (6.1676) Reg loss: 0.4111 (0.6550)
Epoch: [0][8][70] Train loss: 7.3162 (6.1607) Reg loss: 0.6738 (0.6534)
Epoch: [0][8][80] Train loss: 3.5790 (6.1518) Reg loss: 0.31

Epoch: [0][14][60] Train loss: 3.2056 (5.0972) Reg loss: 0.1727 (0.4710)
Epoch: [0][14][70] Train loss: 4.2015 (5.0889) Reg loss: 0.1340 (0.4694)
Epoch: [0][14][80] Train loss: 2.5823 (5.0805) Reg loss: 0.1402 (0.4680)
Epoch: [0][14][90] Train loss: 2.3218 (5.0732) Reg loss: 0.0947 (0.4665)
Epoch: [0][14][100] Train loss: 2.8918 (5.0662) Reg loss: 0.1236 (0.4651)
Epoch: [0][14][110] Train loss: 3.0595 (5.0598) Reg loss: 0.1615 (0.4637)
Epoch: [0][14][120] Train loss: 3.6451 (5.0526) Reg loss: 0.1168 (0.4622)
Epoch: [0][14][130] Train loss: 4.1597 (5.0456) Reg loss: 0.1475 (0.4607)
Epoch: [0][14][140] Train loss: 3.4890 (5.0418) Reg loss: 0.1314 (0.4592)
Epoch: [0][14][150] Train loss: 4.1124 (5.0347) Reg loss: 0.1067 (0.4577)
Epoch: [0][14][160] Train loss: 1.5829 (5.0258) Reg loss: 0.0835 (0.4563)
Epoch: [0][15][0] Train loss: 2.7648 (5.0239) Reg loss: 0.1041 (0.4560)
Epoch: [0][15][10] Train loss: 2.5771 (5.0150) Reg loss: 0.1500 (0.4545)
Epoch: [0][15][20] Train loss: 2.8279 (5.0051

Epoch: [0][20][160] Train loss: 1.3203 (4.2193) Reg loss: 0.0321 (0.3415)
Epoch: [0][21][0] Train loss: 2.0344 (4.2176) Reg loss: 0.0365 (0.3413)
Epoch: [0][21][10] Train loss: 3.4842 (4.2121) Reg loss: 0.0516 (0.3404)
Epoch: [0][21][20] Train loss: 1.3918 (4.2045) Reg loss: 0.0273 (0.3394)
Epoch: [0][21][30] Train loss: 1.9407 (4.1973) Reg loss: 0.0483 (0.3385)
Epoch: [0][21][40] Train loss: 1.3843 (4.1899) Reg loss: 0.0350 (0.3376)
Epoch: [0][21][50] Train loss: 2.2896 (4.1826) Reg loss: 0.0523 (0.3367)
Epoch: [0][21][60] Train loss: 1.3224 (4.1758) Reg loss: 0.0320 (0.3358)
Epoch: [0][21][70] Train loss: 1.6816 (4.1689) Reg loss: 0.0443 (0.3349)
Epoch: [0][21][80] Train loss: 1.4159 (4.1617) Reg loss: 0.0250 (0.3340)
Epoch: [0][21][90] Train loss: 1.7351 (4.1552) Reg loss: 0.0345 (0.3331)
Epoch: [0][21][100] Train loss: 1.6367 (4.1478) Reg loss: 0.0336 (0.3322)
Epoch: [0][21][110] Train loss: 1.4192 (4.1408) Reg loss: 0.0307 (0.3314)
Epoch: [0][21][120] Train loss: 1.7189 (4.1333) R

Epoch: [1][3][100] Train loss: 0.8426 (1.3254) Reg loss: 0.0190 (0.0247)
Epoch: [1][3][110] Train loss: 1.3588 (1.3211) Reg loss: 0.0218 (0.0247)
Epoch: [1][3][120] Train loss: 0.9391 (1.3157) Reg loss: 0.0180 (0.0246)
Epoch: [1][3][130] Train loss: 1.4339 (1.3095) Reg loss: 0.0261 (0.0245)
Epoch: [1][3][140] Train loss: 1.3288 (1.3029) Reg loss: 0.0124 (0.0244)
Epoch: [1][3][150] Train loss: 0.8139 (1.2981) Reg loss: 0.0121 (0.0243)
Epoch: [1][3][160] Train loss: 0.3458 (1.2944) Reg loss: 0.0121 (0.0243)
Epoch: [1][4][0] Train loss: 1.0520 (1.2931) Reg loss: 0.0178 (0.0243)
Epoch: [1][4][10] Train loss: 1.6025 (1.2895) Reg loss: 0.0243 (0.0242)
Epoch: [1][4][20] Train loss: 1.5917 (1.2870) Reg loss: 0.0212 (0.0240)
Epoch: [1][4][30] Train loss: 0.6054 (1.2832) Reg loss: 0.0169 (0.0240)
Epoch: [1][4][40] Train loss: 1.2940 (1.2828) Reg loss: 0.0197 (0.0239)
Epoch: [1][4][50] Train loss: 1.5091 (1.2789) Reg loss: 0.0242 (0.0239)
Epoch: [1][4][60] Train loss: 1.7257 (1.2748) Reg loss: 0.

...

Epoch: [29][3][120] Train loss: 0.0317 (0.0307) Reg loss: 0.0033 (0.0038)
Epoch: [29][3][130] Train loss: 0.0042 (0.0304) Reg loss: 0.0023 (0.0038)
Epoch: [29][3][140] Train loss: 0.0030 (0.0300) Reg loss: 0.0028 (0.0038)
Epoch: [29][3][150] Train loss: 0.0027 (0.0296) Reg loss: 0.0027 (0.0037)
Epoch: [29][3][160] Train loss: 0.0334 (0.0299) Reg loss: 0.0034 (0.0038)
Epoch: [29][4][0] Train loss: 0.0040 (0.0298) Reg loss: 0.0022 (0.0038)
Epoch: [29][4][10] Train loss: 0.0637 (0.0296) Reg loss: 0.0055 (0.0038)
Epoch: [29][4][20] Train loss: 0.0147 (0.0293) Reg loss: 0.0033 (0.0037)
Epoch: [29][4][30] Train loss: 0.0192 (0.0291) Reg loss: 0.0041 (0.0037)
Epoch: [29][4][40] Train loss: 0.0658 (0.0290) Reg loss: 0.0032 (0.0037)
Epoch: [29][4][50] Train loss: 0.0039 (0.0287) Reg loss: 0.0025 (0.0037)
Epoch: [29][4][60] Train loss: 0.0169 (0.0284) Reg loss: 0.0028 (0.0037)
Epoch: [29][4][70] Train loss: 0.0036 (0.0281) Reg loss: 0.0014 (0.0037)
Epoch: [29][4][80] Train loss: 0.0222 (0.0278) 

Epoch: [29][10][50] Train loss: 0.0068 (0.0362) Reg loss: 0.0027 (0.0043)
Epoch: [29][10][60] Train loss: 0.0464 (0.0363) Reg loss: 0.0076 (0.0043)
Epoch: [29][10][70] Train loss: 0.0032 (0.0362) Reg loss: 0.0030 (0.0043)
Epoch: [29][10][80] Train loss: 0.0306 (0.0361) Reg loss: 0.0042 (0.0043)
Epoch: [29][10][90] Train loss: 0.0135 (0.0360) Reg loss: 0.0030 (0.0043)
Epoch: [29][10][100] Train loss: 0.0013 (0.0359) Reg loss: 0.0037 (0.0043)
Epoch: [29][10][110] Train loss: 0.0127 (0.0359) Reg loss: 0.0041 (0.0043)
Epoch: [29][10][120] Train loss: 0.0053 (0.0359) Reg loss: 0.0019 (0.0043)
Epoch: [29][10][130] Train loss: 0.0132 (0.0358) Reg loss: 0.0031 (0.0043)
Epoch: [29][10][140] Train loss: 0.0261 (0.0357) Reg loss: 0.0036 (0.0043)
Epoch: [29][10][150] Train loss: 0.0008 (0.0356) Reg loss: 0.0021 (0.0043)
Epoch: [29][10][160] Train loss: 0.0280 (0.0354) Reg loss: 0.0029 (0.0043)
Epoch: [29][11][0] Train loss: 0.0022 (0.0354) Reg loss: 0.0023 (0.0043)
Epoch: [29][11][10] Train loss: 

Epoch: [29][16][140] Train loss: 0.0138 (0.0328) Reg loss: 0.0029 (0.0040)
Epoch: [29][16][150] Train loss: 0.0063 (0.0328) Reg loss: 0.0032 (0.0040)
Epoch: [29][16][160] Train loss: 0.0280 (0.0327) Reg loss: 0.0042 (0.0040)
Epoch: [29][17][0] Train loss: 0.0222 (0.0327) Reg loss: 0.0044 (0.0040)
Epoch: [29][17][10] Train loss: 0.0886 (0.0328) Reg loss: 0.0066 (0.0040)
Epoch: [29][17][20] Train loss: 0.0117 (0.0327) Reg loss: 0.0033 (0.0040)
Epoch: [29][17][30] Train loss: 0.0049 (0.0326) Reg loss: 0.0027 (0.0040)
Epoch: [29][17][40] Train loss: 0.0052 (0.0329) Reg loss: 0.0021 (0.0040)
Epoch: [29][17][50] Train loss: 0.0148 (0.0329) Reg loss: 0.0028 (0.0040)
Epoch: [29][17][60] Train loss: 0.0031 (0.0328) Reg loss: 0.0027 (0.0040)
Epoch: [29][17][70] Train loss: 0.0079 (0.0330) Reg loss: 0.0027 (0.0040)
Epoch: [29][17][80] Train loss: 0.0605 (0.0330) Reg loss: 0.0045 (0.0040)
Epoch: [29][17][90] Train loss: 0.0199 (0.0329) Reg loss: 0.0030 (0.0040)
Epoch: [29][17][100] Train loss: 0.0

Epoch: [29][23][60] Train loss: 0.0220 (0.0338) Reg loss: 0.0026 (0.0041)
Epoch: [29][23][70] Train loss: 0.0069 (0.0337) Reg loss: 0.0026 (0.0041)
Epoch: [29][23][80] Train loss: 0.0993 (0.0336) Reg loss: 0.0126 (0.0041)
Epoch: [29][23][90] Train loss: 0.0036 (0.0337) Reg loss: 0.0019 (0.0041)
Epoch: [29][23][100] Train loss: 0.0160 (0.0337) Reg loss: 0.0023 (0.0041)
Epoch: [29][23][110] Train loss: 0.0088 (0.0338) Reg loss: 0.0020 (0.0041)
Epoch: [29][23][120] Train loss: 0.0460 (0.0337) Reg loss: 0.0024 (0.0040)
Epoch: [29][23][130] Train loss: 0.0046 (0.0337) Reg loss: 0.0029 (0.0040)
Epoch: [29][23][140] Train loss: 0.0223 (0.0336) Reg loss: 0.0025 (0.0040)
Epoch: [29][23][150] Train loss: 0.0012 (0.0336) Reg loss: 0.0018 (0.0040)
Epoch: [29][23][160] Train loss: 0.0119 (0.0336) Reg loss: 0.0017 (0.0040)
Epoch: [29][24][0] Train loss: 0.0016 (0.0335) Reg loss: 0.0019 (0.0040)
Epoch: [29][24][10] Train loss: 0.0138 (0.0335) Reg loss: 0.0018 (0.0040)
Epoch: [29][24][20] Train loss: 

Epoch: [30][5][160] Train loss: 0.0063 (0.0245) Reg loss: 0.0024 (0.0032)
Epoch: [30][6][0] Train loss: 0.0059 (0.0244) Reg loss: 0.0020 (0.0032)
Epoch: [30][6][10] Train loss: 0.0149 (0.0243) Reg loss: 0.0018 (0.0031)
Epoch: [30][6][20] Train loss: 0.0034 (0.0242) Reg loss: 0.0017 (0.0031)
Epoch: [30][6][30] Train loss: 0.0115 (0.0240) Reg loss: 0.0030 (0.0031)
Epoch: [30][6][40] Train loss: 0.0262 (0.0239) Reg loss: 0.0029 (0.0031)
Epoch: [30][6][50] Train loss: 0.0088 (0.0237) Reg loss: 0.0019 (0.0031)
Epoch: [30][6][60] Train loss: 0.0027 (0.0240) Reg loss: 0.0018 (0.0031)
Epoch: [30][6][70] Train loss: 0.0073 (0.0238) Reg loss: 0.0015 (0.0031)
Epoch: [30][6][80] Train loss: 0.0680 (0.0237) Reg loss: 0.0078 (0.0031)
Epoch: [30][6][90] Train loss: 0.0023 (0.0237) Reg loss: 0.0021 (0.0031)
Epoch: [30][6][100] Train loss: 0.0061 (0.0236) Reg loss: 0.0022 (0.0031)
Epoch: [30][6][110] Train loss: 0.0033 (0.0234) Reg loss: 0.0023 (0.0031)
Epoch: [30][6][120] Train loss: 0.0035 (0.0237) R